### Exercise project 3 – Support Vector Machines
#### In this project I will use SVR = Support Vector Regression (type of
### SVM for regression). I will use my previous regression dataset "California Housing Prices" 
##### (https://www.kaggle.com/datasets/camnugent/california-housing-prices). 
#### Here will be used the same dataset optimizations as in Project_1.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics, svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler